In [ ]:
from helpers.readSetFiles import readSetFiles,validateParameters,readNumberSetFiles
from graph import plotEpochsError,plotGraphEj1,plotGraphEj1Animated,plotErrorAgainstKGraph,plotErrorAgainstTrainingPercentageGraph,plotMetricsAgainstVariable,plotMetricAgainstVariable
from helpers.configHelper import ConfigHelper
from neuralNetworkManager import NeuralNetworkManager
from activationFunctions.stepActivationFunction import StepActivationFunction
from activationFunctions.linealActivationFunction import LinealActivationFunction
from activationFunctions.hyperbolicTangentActivationFunction import HyperbolicTangentActivationFunction
from activationFunctions.sigmoidalActivationFunction import SigmoidalActivationFunction
from app import normalize
from output import Output
from metrics import Metrics
import copy

# Ej1 - Perceptron Simple Escalon

##### Funcion logica ’Y’ con entradas

In [ ]:
configPath="./config/Ej1/configEj1.json"
configHelper = ConfigHelper(configPath)
(architecture,activationFunction,beta,learningRate,maxIterations,maxToleranceExponent) = configHelper.getProperties()
(trainingSet,resultsSet)=readSetFiles("./datasets/Ej1/training_set_ej_1.txt","./datasets/Ej1/result_set_ej_1_and.txt")

neuralNetworkManager = NeuralNetworkManager(architecture,activationFunction,learningRate,maxIterations,maxToleranceExponent)
(epochs,executionTime,exception) = neuralNetworkManager.start(trainingSet,resultsSet)
#Plot error graph
plotEpochsError(epochs)
#Plot points and division graph
isXOR = False
plotGraphEj1(trainingSet,isXOR,epochs)
##Crear la carpeta de resultados con la subcarpeta de graficos y estadisticas en caso de que no existan
isEj1 = True
plotGraphEj1Animated(trainingSet,isXOR,epochs,isEj1)
#Print output
print("FINISH-------------------------------------------------------------------------------------------")
output = Output(configHelper,epochs[-1].error,epochs[-1].iterationNumber,executionTime)
output.printOutput()

##### Funcion logica ’O exclusivo’ con entradas

In [ ]:
configPath="./config/Ej1/configEj1.json"
configHelper = ConfigHelper(configPath)
(architecture,activationFunction,beta,learningRate,maxIterations,maxToleranceExponent) = configHelper.getProperties()
(trainingSet,resultsSet)=readSetFiles("./datasets/Ej1/training_set_ej_1.txt","./datasets/Ej1/result_set_ej_1_xor.txt")

neuralNetworkManager = NeuralNetworkManager(architecture,activationFunction,learningRate,maxIterations,maxToleranceExponent)
(epochs,executionTime,exception) = neuralNetworkManager.start(trainingSet,resultsSet)
#Plot error graph
plotEpochsError(epochs)
#Plot points and division graph
isXOR = True
plotGraphEj1(trainingSet,isXOR,epochs)
isEj1 = True
plotGraphEj1Animated(trainingSet,isXOR,epochs,isEj1)
#Print output
print("FINISH-------------------------------------------------------------------------------------------")
output = Output(configHelper,epochs[-1].error,epochs[-1].iterationNumber,executionTime)
output.printOutput()

# Ej2 Perceptron Simple lineal/no lineal


In [ ]:
(architecture,activationFunction,beta,learningRate,maxIterations,maxToleranceExponent) = ConfigHelper("./config/Ej2/configEj2.json").getProperties()
(trainingSet,resultsSet)=readSetFiles("./datasets/Ej2/training_set_ej_2.txt","./datasets/Ej2/result_set_ej_2.txt")
(resultsSetNormalized,fixnt)=normalize(resultsSet,0,1)
activationFunctions=[HyperbolicTangentActivationFunction(beta),SigmoidalActivationFunction(beta)]
neuralNetworkManager = NeuralNetworkManager(architecture,LinealActivationFunction.getType(beta),learningRate,maxIterations,maxToleranceExponent)
(epochs,executionTime,exception) = neuralNetworkManager.start(trainingSet,resultsSet)
print("----------------------EVALUATE GENERALITAZTION----------------------------------------------------")
errors = []
kArray = []
trainingPercentages = []
for k in range(2,len(trainingSet)+1):
    kArray.append(k)
    (error,trainingPercentage,maxAccuracy) = neuralNetworkManager.crossValidation(trainingSet,resultsSet,k)
    errors.append(error)
    trainingPercentages.append(trainingPercentage)
plotErrorAgainstTrainingPercentageGraph(errors,trainingPercentages)
plotEpochsError(epochs,"Lineal Activation Function")
for aF in activationFunctions:
    neuralNetworkManager = NeuralNetworkManager(architecture,aF,learningRate,maxIterations,maxToleranceExponent)
    (epochs,executionTime,exception) = neuralNetworkManager.start(trainingSet,resultsSetNormalized)
    plotEpochsError(epochs,title=aF.name,denormalizeFn=fixnt)
    errors = []
    kArray = []
    trainingPercentages = []
    for k in range(2,len(trainingSet)+1):
        kArray.append(k)
        (error,trainingPercentage,maxAccuracy) = neuralNetworkManager.crossValidation(trainingSet,resultsSetNormal,k)
        errors.append(error)
        trainingPercentages.append(trainingPercentage)
    plotErrorAgainstTrainingPercentageGraph(errors,trainingPercentages)


In [ ]:
configHelper = ConfigHelper("./config/Ej2/configEj2.json")
(architecture,activationFunction,beta,learningRate,maxIterations,maxToleranceExponent) = configHelper.getProperties()
(trainingSet,resultsSet)=readSetFiles("./datasets/Ej2/training_set_ej_2.txt","./datasets/Ej2/result_set_ej_2.txt")

In [ ]:
#LINEAL
activationFunctions=[HyperbolicTangentActivationFunction(beta),SigmoidalActivationFunction(beta)]
neuralNetworkManager = NeuralNetworkManager(architecture,LinealActivationFunction.getType(beta),learningRate,maxIterations,maxToleranceExponent)
#Training
(epochs,executionTime,exception) = neuralNetworkManager.start(trainingSet,resultsSet)
print("----------------------FINISH TRAINING----------------------------------------------------")
output = Output(configHelper,epochs[-1].error,epochs[-1].iterationNumber,executionTime)
output.printOutput()
#Testing
print("----------------------EVALUATE GENERALITAZTION----------------------------------------------------")
errors = []
kArray = []
trainingPercentages = []
for k in range(2,len(trainingSet)+1):
    kArray.append(k)
    (error,trainingPercentage,maxAccuracy) = neuralNetworkManager.crossValidation(trainingSet,resultsSet,k)
    errors.append(error)
    trainingPercentages.append(trainingPercentage)
plotErrorAgainstKGraph(errors,kArray) 
plotErrorAgainstTrainingPercentageGraph(errors,trainingPercentages)
plotEpochsError(epochs,"Lineal Activation Function")

In [14]:
#TANH
activationFunctions=[HyperbolicTangentActivationFunction(beta),SigmoidalActivationFunction(beta)]
neuralNetworkManager = NeuralNetworkManager(architecture,HyperbolicTangentActivationFunction.getType(beta),learningRate,maxIterations,maxToleranceExponent)
#Normalize
(resultsSetNormalized,fixnt)=normalize(resultsSet,-1,1)
#Training
(epochs,executionTime,exception) = neuralNetworkManager.start(trainingSet,resultsSetNormalized)
print("----------------------FINISH TRAINING----------------------------------------------------")
output = Output(configHelper,epochs[-1].error,epochs[-1].iterationNumber,executionTime)
output.printOutput()
#Testing
print("----------------------EVALUATE GENERALITAZTION----------------------------------------------------")
errors = []
kArray = []
trainingPercentages = []
for k in range(2,len(trainingSet)+1):
    kArray.append(k)
    (error,trainingPercentage,maxAccuracy) = neuralNetworkManager.crossValidation(trainingSet,resultsSetNormalized,k)
    errors.append(error)
    trainingPercentages.append(trainingPercentage)
plotErrorAgainstKGraph(errors,kArray) 
plotErrorAgainstTrainingPercentageGraph(errors,trainingPercentages)
plotEpochsError(epochs,"Tanh Activation Function")

----------------------FINISH TRAINING----------------------------------------------------
OUTPUT :
        - Configuration paramaters :
            	-Architecture : [3, 1]
		-Activation function :LINEAL
		-Beta : 1
		-Learning rate : 0.1 
		-Max iterations : 1000
		-Error bound : 1e^-5
        - Error : 1360.1241083186706

        - Epoch : 999

        - Execution time : 47.100314600000274 sec
----------------------EVALUATE GENERALITAZTION----------------------------------------------------
Cross validation with k=2 results:
	error min == 1279.0833686874748
	training percentage == 0.5


In [ ]:
#SIGMOIDAL
neuralNetworkManager = NeuralNetworkManager(architecture,SigmoidalActivationFunction.getType(beta),learningRate,maxIterations,maxToleranceExponent)
#Normalize
(resultsSetNormalized,fixnt)=normalize(resultsSet,0,1)
#Training
(epochs,executionTime,exception) = neuralNetworkManager.start(trainingSet,resultsSetNormalized)
print("----------------------FINISH TRAINING----------------------------------------------------")
output = Output(configHelper,epochs[-1].error,epochs[-1].iterationNumber,executionTime)
output.printOutput()
#Testing
print("----------------------EVALUATE GENERALITAZTION----------------------------------------------------")
errors = []
kArray = []
trainingPercentages = []
for k in range(2,len(trainingSet)+1):
    kArray.append(k)
    (error,trainingPercentage,maxAccuracy) = neuralNetworkManager.crossValidation(trainingSet,resultsSetNormalized,k)
    errors.append(error)
    trainingPercentages.append(trainingPercentage)
plotErrorAgainstKGraph(errors,kArray) 
plotErrorAgainstTrainingPercentageGraph(errors,trainingPercentages)
plotEpochsError(epochs,"Sigmoidal Activation Function")

# Ej3 Perceptron Multicapa

##### 3.1 Funciones logicas ’Y’ y ’O exclusivo’ con entradas

##### ’Y’

In [ ]:
import random
import numpy as np
random.seed(11)
np.random.seed(11)
configPath="./config/Ej3/3_1/configEj3_1.json"
configHelper = ConfigHelper(configPath)
(architecture,activationFunction,beta,learningRate,maxIterations,maxToleranceExponent) = configHelper.getProperties()
(trainingSet,resultsSet)=readSetFiles("./datasets/Ej1/training_set_ej_1.txt","./datasets/Ej1/result_set_ej_1_and.txt")

neuralNetworkManager = NeuralNetworkManager(architecture,activationFunction,learningRate,maxIterations,maxToleranceExponent)
(epochs,executionTime,exception) = neuralNetworkManager.start(trainingSet,resultsSet)
#Plot error graph
plotEpochsError(epochs)
#Plot points and division graph
isXOR = False
plotGraphEj1(trainingSet,isXOR,epochs)
isEj1 = False
plotGraphEj1Animated(trainingSet,isXOR,epochs,isEj1)
#Print output
print("FINISH-------------------------------------------------------------------------------------------")
output = Output(configHelper,epochs[-1].error,epochs[-1].iterationNumber,executionTime)
output.printOutput()

##### ’O exclusivo’

In [ ]:
configPath="./config/Ej3/3_1/configEj3_1.json"
configHelper = ConfigHelper(configPath)
(architecture,activationFunction,beta,learningRate,maxIterations,maxToleranceExponent) = configHelper.getProperties()
(trainingSet,resultsSet)=readSetFiles("./datasets/Ej1/training_set_ej_1.txt","./datasets/Ej1/result_set_ej_1_and.txt")

neuralNetworkManager = NeuralNetworkManager(architecture,activationFunction,learningRate,maxIterations,maxToleranceExponent)
(epochs,executionTime,exception) = neuralNetworkManager.start(trainingSet,resultsSet)
#Plot error graph
plotEpochsError(epochs)
#Plot points and division graph
isXOR = True
plotGraphEj1(trainingSet,isXOR,epochs)
isEj1 = False
plotGraphEj1Animated(trainingSet,isXOR,epochs,isEj1)
#Print output
print("FINISH-------------------------------------------------------------------------------------------")
output = Output(configHelper,epochs[-1].error,epochs[-1].iterationNumber,executionTime)
output.printOutput()

##### 3.2 Indicar paridad para numeros del 0 al 9

In [ ]:
configPath="./config/Ej3/3_2/configEj3_2.json"
configHelper = ConfigHelper(configPath)
(architecture,activationFunction,beta,learningRate,maxIterations,maxToleranceExponent) = configHelper.getProperties()
(trainingSet,resultsSet)=readNumberSetFiles("./datasets/Ej3/training_set_ej_3.txt","./datasets/Ej3/result_set_ej_3_2.txt")
print(trainingSet)
print(resultsSet)
#Creamos la instancia para metricas
classes = [1,-1]
metrics = Metrics(classes,False,-1,1)
neuralNetworkManager = NeuralNetworkManager(architecture,activationFunction,learningRate,maxIterations,maxToleranceExponent)
(epochs,executionTime,exception) = neuralNetworkManager.start(trainingSet,resultsSet)
print("FINISH-------------------------------------------------------------------------------------------")
# print(epochs)
output = Output(configHelper,epochs[-1].error,epochs[-1].iterationNumber,executionTime)
output.printOutput()
print("----------------------EVALUATE GENERALITAZTION----------------------------------------------------")
errors = []
kArray = []
trainingPercentages = []
# metricsArrays = {'accuracy':[],'precision':[],'recall':[],'f_1-score':[],'true_positive_rate':[],'false_positive_rate':[]}
accuracyArray = []
for k in range(2,len(trainingSet)+1):
    kArray.append(k)
    (error,trainingPercentage,maxAccuracy) = neuralNetworkManager.crossValidation(trainingSet,resultsSet,k,copy.deepcopy(metrics))
    errors.append(error)
    trainingPercentages.append(trainingPercentage)
    accuracyArray.append(maxAccuracy)
    # metricsArrays['accuracy'].append(metricsDict['accuracy'])
    # metricsArrays['precision'].append(metricsDict['precision'])
    # metricsArrays['recall'].append(metricsDict['recall'])
    # metricsArrays['f_1-score'].append(metricsDict['f_1-score'])
    # metricsArrays['true_positive_rate'].append(metricsDict['true_positive_rate'])
    # metricsArrays['false_positive_rate'].append(metricsDict['false_positive_rate'])
# plotMetricsAgainstVariable(metricsArrays,kArray,"K")
# plotMetricsAgainstVariable(metricsArrays,trainingPercentages,"Training percentage")
plotMetricAgainstVariable(accuracyArray,"Accuracy",kArray,"K")
plotMetricAgainstVariable(accuracyArray,"Accuracy",trainingPercentages,"Training percentage")


##### 3.3 Indicar que numero es, para numeros del 0 al 9

In [ ]:
configPath="./config/Ej3/3_3/configEj3_3.json"
configHelper = ConfigHelper(configPath)
(architecture,activationFunction,beta,learningRate,maxIterations,maxToleranceExponent) = configHelper.getProperties()
(trainingSet,resultsSet)=readNumberSetFiles("./datasets/Ej3/training_set_ej_3.txt","./datasets/Ej3/result_set_ej_3_3.txt")
print(trainingSet)
print(resultsSet)
#Creamos la instancia para metricas
classes = [0,1,2,3,4,5,6,7,8,9]
metrics = Metrics(classes,True,-1,1)
neuralNetworkManager = NeuralNetworkManager(architecture,activationFunction,learningRate,maxIterations,maxToleranceExponent)
(epochs,executionTime,exception) = neuralNetworkManager.start(trainingSet,resultsSet)
print("FINISH-------------------------------------------------------------------------------------------")
# print(epochs)
output = Output(configHelper,epochs[-1].error,epochs[-1].iterationNumber,executionTime)
output.printOutput()
print("----------------------EVALUATE GENERALITAZTION----------------------------------------------------")
errors = []
kArray = []
trainingPercentages = []
# metricsArrays = {'accuracy':[],'precision':[],'recall':[],'f_1-score':[],'true_positive_rate':[],'false_positive_rate':[]}
accuracyArray = []
for k in range(2,len(trainingSet)+1):
    kArray.append(k)
    (error,trainingPercentage,maxAccuracy) = neuralNetworkManager.crossValidation(trainingSet,resultsSet,k,copy.deepcopy(metrics))
    errors.append(error)
    trainingPercentages.append(trainingPercentage)
    accuracyArray.append(maxAccuracy)
    # metricsArrays['accuracy'].append(metricsDict['accuracy'])
    # metricsArrays['precision'].append(metricsDict['precision'])
    # metricsArrays['recall'].append(metricsDict['recall'])
    # metricsArrays['f_1-score'].append(metricsDict['f_1-score'])
    # metricsArrays['true_positive_rate'].append(metricsDict['true_positive_rate'])
    # metricsArrays['false_positive_rate'].append(metricsDict['false_positive_rate'])
plotMetricAgainstVariable(accuracyArray,"Accuracy",kArray,"K")
plotMetricAgainstVariable(accuracyArray,"Accuracy",trainingPercentages,"Training percentage")